## Trabalho Prático 2 - Análise e Tratamento de Imagens e Vídeos Digitais

In [3]:
import cv2 as cv
import numpy as np
from multiprocessing import Pool
import warnings
import time

### Quadros de corte de cena reais em cada vídeo

In [4]:
cortes_reais = {'titanic': [124, 332, 376, 513, 563, 636, 669, 685, 723, 788, 819, 920, 966, 1000, 
                           1123, 1384, 1611, 1667, 1820, 2138, 2223, 2305, 2615, 2889, 2990, 3109, 3659],
                'pica_pau':[304, 914, 940, 1294, 1361, 1406, 1504, 1594, 1819, 1901, 2216, 2260, 
                            2341, 2429, 2628, 2704, 2804, 2860, 2929, 3004, 3160, 3530, 3604,
                            3654, 3892, 4010, 4260, 4366, 4541, 4616, 4729, 4778],
                'jornal' : [157, 458, 554, 1423, 1454, 1621, 1701],
                'will_smith': [259, 512, 728, 1648, 1901, 2172, 2357, 2402, 2530, 3098, 3136, 
                               3411, 3738, 4595, 4602, 4640, 4665, 4693, 4771, 4799, 4877, 4906, 
                               5013, 5039, 5091, 5130, 5179, 5213, 5303, 5373, 5490, 5502, 5518, 5573],
                'homem_de_ferro':[33, 98, 136, 231, 242, 300, 327, 350, 356, 381, 404, 433, 449, 483, 503, 
                                 582, 601, 647, 659, 683, 699, 714, 738, 756, 786, 812, 837, 856, 871, 904,
                                 918, 953, 984, 1043, 1070, 1114, 1131, 1147, 1158, 1185, 1192, 1214, 1229,
                                 1245, 1259, 1268, 1279,1288, 1296, 1312, 1337, 1360, 1385, 1401, 1419, 1437,
                                 1449, 1458, 1470, 1510, 1523, 1595, 1639, 1666, 1679, 1701, 1724, 1775, 1800,
                                 1833, 1941, 2023, 2117, 2212, 2276, 2315, 2520, 2558, 2686, 2708],
                'mario_kart_tour': [31,  213, 262, 308, 361, 410, 459, 508, 559, 606, 708, 804, 901, 1001, 1101,
                                  1198, 1289, 1394, 1445, 1496, 1545, 1592, 1693, 1735, 1772, 1827, 1854, 1939,
                                  1988, 2058, 2081, 2113, 2145, 2170, 2199, 2221, 2245, 2276, 2295, 2316, 2333,
                                  2457, 2732, 2741],
                'pontos_federer': [20, 154, 178, 436, 668, 821, 927, 936, 947, 1039, 1486, 1553, 1765, 1887, 2949],
                'coracao_valente': [23, 37, 108, 197, 338, 459, 528, 562, 587, 643, 708, 742, 787, 820, 982, 1100, 
                                   1125, 1237, 1315, 1378, 1436, 1470, 1512, 1570, 1625, 1671, 1733, 1877, 1942, 
                                   2073, 2120, 2400, 2428, 2451, 2792, 2914, 3235, 3291, 3311, 3343, 3367, 3451, 
                                   3676, 3709, 3743, 3845, 3936, 4081, 4112, 4151, 4317, 4378, 4458, 4506, 4554, 
                                   4579, 4618, 4659, 4699, 4750, 4841, 4883, 4940, 4959, 4972, 5016, 5030],
                'warcraft_trailler': [120, 153, 178, 196, 236, 252, 285, 300, 362, 400, 430, 470, 504, 547, 579, 
                                     621, 628, 672, 702, 717, 727, 740, 760, 768, 782, 802, 874, 899, 925, 952, 
                                     1003, 1039, 1080, 1115, 1147, 1198, 1222, 1262, 1304, 1340, 1353, 1375, 1393, 
                                     1408, 1439, 1470, 1494, 1521, 1562, 1580, 1606, 1624, 1636, 1661, 1715, 1758, 
                                     1775, 1800, 1829, 1867, 1894, 1959, 1967, 1987, 2004, 2024, 2040, 2084, 2112, 
                                     2119, 2136, 2160, 2188, 2219, 2239, 2265, 2273, 2291, 2304, 2326, 2358, 2370, 
                                     2382, 2397, 2430, 2448, 2477, 2491, 2508, 2521, 2542, 2556, 2577, 2596, 2625, 
                                     2678, 2716, 2730, 2761, 2819, 2860, 2900, 3100, 3114, 3185], 
                'rei_leao': [30, 679, 801, 976, 1027, 1087, 1251, 1301, 1351, 1392, 1563, 1669, 1779, 1815, 1897, 2088,
                            2279, 2371, 2508, 2535, 2565, 2632, 2728, 2847, 3122, 3229, 3286, 3413, 3649, 3711, 3997, 
                            4117, 4536, 4689], 
               }

# Número de quadros dos vídeos:
# reiLeao = 30 ; warcraft_trailler = 24 ; coracao_calente = 25 ; pontos_federer = 25 ; mario_kart_tour = 30 ; 
# homem_de_ferro = 24 ; will_smith = 30 ; jornal = 24 ; pica_pau = 30 ; titanic = 24;

## Funções utilitárias

In [5]:
def calcular_acuracia(quadros_de_corte_reais, quadros_de_corte_preditos, fps):
    quadros_reais_identificados = []
    if(len(quadros_de_corte_preditos) == 0):
        return None
    conta_predicoes_corretas = 0
    predicoes_corretas = []
    predicoes_erradas = []
    for quadro_predito in quadros_de_corte_preditos:
        is_predicao_correta = False
        for quadro_real in quadros_de_corte_reais:
            if((quadro_real - fps <= quadro_predito) and (quadro_predito <= quadro_real + fps)):
                is_predicao_correta = True
                if(not quadro_real in quadros_reais_identificados):
                    predicoes_corretas.append(quadro_predito)
                    conta_predicoes_corretas += 1
                    quadros_reais_identificados.append(quadro_real)
                    break
            elif(quadro_real + fps > quadro_predito):
                break
        if(not is_predicao_correta):
            predicoes_erradas.append(quadro_predito)
                        
    acuracia = conta_predicoes_corretas / len(quadros_de_corte_reais)
    return acuracia, predicoes_corretas, predicoes_erradas

def mostrar_resultados_da_deteccao_de_cenas(quadros_de_corte_reais, quadros_de_corte_preditos, fps):
    resultado = calcular_acuracia(quadros_de_corte_reais, quadros_de_corte_preditos, fps)
    if(resultado == None):
        print('nenhum quadro de corte foi predito')
        return
    print('\n','quadros reais:', quadros_de_corte_reais, '\n')
    print('acurácia: {0:.2f}'.format(resultado[0]), '\n')
    print('quadros preditos corretamente:', resultado[1], '\n')
    print('quadros preditos incorretamente:', resultado[2])
    return resultado[0]

def soma_das_diferencas(histograma1, histograma2):
    return np.sum(abs(histograma1[:] - histograma2[:]))

def dLog_em_pixel(valor):
    if(valor > 1):
        return np.ceil(np.log2(valor)) + 1
    elif(valor == 0):
        return 0
    else:
        return 1
    
def dLog(histograma1, histograma2):
    distancia = 0
    for i in range(len(histograma1)):
        distancia += abs(dLog_em_pixel(histograma1[i]) - dLog_em_pixel(histograma2[i]))
    return distancia

def existe_corte_entre_bgr_histogramas(histogramas1, histogramas2, distancia, limiar):
    if(distancia(histogramas1[0], histogramas2[0]) > limiar
       or distancia(histogramas1[1], histogramas2[1]) > limiar
       or distancia(histogramas1[2], histogramas2[2]) > limiar):
        return True
    else:
        return False

## 1. Histograma global

In [6]:
def histograma_global(img):
    histograma_azul = np.zeros(256, dtype = int)
    histograma_verde = np.zeros(256, dtype = int)
    histograma_vermelho = np.zeros(256, dtype = int)

    for i in range(0, len(img)):
        for j in range(0,len(img[1])):
            bgr = img[i][j]
            histograma_azul[bgr[0]] +=1
            histograma_verde[bgr[1]] +=1
            histograma_vermelho[bgr[2]] +=1

    return histograma_azul, histograma_verde, histograma_vermelho

def aplicar_histograma_global_em_video(video, fps, quadro_limite=None):
    proximo_frame = (fps + 1) // 2
    histogramas = []
    cap = cv.VideoCapture(video)
    total_de_frames = cap.get(7)
    
    cap.set(1, 1)
    ret, frame = cap.read()
    progresso = 0
    while(proximo_frame < total_de_frames):
        cap.set(1, proximo_frame)
        ret, frame = cap.read()
        if(ret == False):
            break
        histogramas.append((proximo_frame, histograma_global(frame)))
        progresso = int(proximo_frame / total_de_frames * 100)
        print(f'{progresso}%', end=' ')
        proximo_frame += fps
        if(quadro_limite != None):
            if(proximo_frame >= quadro_limite):
                cap.release()
                cv.destroyAllWindows()
                print("parou no quadro limite")
                return histogramas
    cap.release()
    cv.destroyAllWindows()
    print('100%')
    return histogramas

def get_quadros_de_corte_do_histograma_global(histogramas, distancia, limiar):
    quadros_de_corte = []
    for k in range(len(histogramas) - 1):
        bgr_histogramas1 = histogramas[k][1]
        bgr_histogramas2 = histogramas[k + 1][1]
        if(existe_corte_entre_bgr_histogramas(bgr_histogramas1, bgr_histogramas2, distancia, limiar)):
            quadros_de_corte.append((histogramas[k][0]))
    return quadros_de_corte

def roda_experimento_histograma_global(video, fps, quadros_de_corte_reais, distancia, limiar, quadro_limite=None):
    histogramas = aplicar_histograma_global_em_video(video, fps, quadro_limite=quadro_limite)
    quadros_de_corte_preditos = get_quadros_de_corte_do_histograma_global(histogramas, distancia, limiar)
    return mostrar_resultados_da_deteccao_de_cenas(quadros_de_corte_reais, quadros_de_corte_preditos, fps)

## Detecção de cenas

In [5]:
limiar_histograma_global = 150000

### Titanic

In [6]:
roda_experimento_histograma_global('titanic-640x360.mp4', 24, cortes_reais['titanic'], 
                                           soma_das_diferencas, limiar_histograma_global)

0% 0% 1% 2% 2% 3% 3% 4% 5% 5% 6% 6% 7% 8% 8% 9% 9% 10% 11% 11% 12% 12% 13% 14% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 21% 22% 23% 23% 24% 24% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 39% 40% 41% 41% 42% 42% 43% 44% 44% 45% 45% 46% 47% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 54% 55% 56% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 63% 64% 65% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 72% 73% 74% 74% 75% 75% 76% 77% 77% 78% 78% 79% 80% 80% 81% 81% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 93% 94% 95% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [124, 332, 376, 513, 563, 636, 669, 685, 723, 788, 819, 920, 966, 1000, 1123, 1384, 1611, 1667, 1820, 2138, 2223, 2305, 2615, 2889, 2990, 3109, 3659] 

acurácia: 0.89 

quadros preditos corretamente: [108, 324, 372, 492, 540, 612, 660, 684, 708, 780, 804, 900, 948, 996, 1380, 1596, 1812, 2124, 2220, 2292, 2604, 2868, 3108, 36

0.8888888888888888

### Pica pau

In [7]:
roda_experimento_histograma_global('pica-pau.mkv', 30, cortes_reais['pica_pau'], 
                                           soma_das_diferencas, limiar_histograma_global)

0% 0% 1% 2% 2% 3% 4% 4% 5% 5% 6% 7% 7% 8% 9% 9% 10% 10% 11% 12% 12% 13% 13% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 28% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 40% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 55% 55% 56% 56% 57% 58% 58% 59% 59% 60% 61% 61% 62% 63% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 74% 75% 76% 76% 77% 78% 78% 79% 79% 80% 81% 81% 82% 82% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 91% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 97% 98% 99% 99% 100%

 quadros reais: [304, 914, 940, 1294, 1361, 1406, 1504, 1594, 1819, 1901, 2216, 2260, 2341, 2429, 2628, 2704, 2804, 2860, 2929, 3004, 3160, 3530, 3604, 3654, 3892, 4010, 4260, 4366, 4541, 4616, 4729, 4778] 

acurácia: 0.69 

quadros preditos corretamente: [285, 885, 915, 1275, 1395, 1485, 1575, 1815, 2205, 2235, 2325, 2415, 2625, 2685, 2925, 3135, 4005, 4245, 4515

0.6875

### Jornal G1

In [8]:
roda_experimento_histograma_global('jornal.mkv', 14, cortes_reais['jornal'], 
                                   soma_das_diferencas, limiar_histograma_global)

0% 1% 1% 2% 3% 4% 4% 5% 6% 7% 8% 8% 9% 10% 11% 11% 12% 13% 14% 14% 15% 16% 17% 17% 18% 19% 20% 21% 21% 22% 23% 24% 24% 25% 26% 27% 27% 28% 29% 30% 30% 31% 32% 33% 34% 34% 35% 36% 37% 37% 38% 39% 40% 40% 41% 42% 43% 43% 44% 45% 46% 47% 47% 48% 49% 50% 50% 51% 52% 53% 53% 54% 55% 56% 56% 57% 58% 59% 60% 60% 61% 62% 63% 63% 64% 65% 66% 66% 67% 68% 69% 70% 70% 71% 72% 73% 73% 74% 75% 76% 76% 77% 78% 79% 79% 80% 81% 82% 83% 83% 84% 85% 86% 86% 87% 88% 89% 89% 90% 91% 92% 92% 93% 94% 95% 96% 96% 97% 98% 99% 99% 100%

 quadros reais: [157, 458, 554, 1423, 1454, 1621, 1701] 

acurácia: 0.71 

quadros preditos corretamente: [147, 455, 553, 1449, 1687] 

quadros preditos incorretamente: [21, 35, 63, 105, 1407, 1603, 1743]


0.7142857142857143

### Entrevista do Will Smith

In [9]:
roda_experimento_histograma_global('will_smith_interview360p.mp4', 30, cortes_reais['will_smith'], 
                                   soma_das_diferencas, limiar_histograma_global)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 10% 10% 11% 11% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [259, 512, 728, 1648, 1901, 2172, 2357, 2402, 2530, 3098, 3136, 3411, 3738, 4595, 4602, 4640, 4665, 4693, 4771, 4799, 4877, 4906, 5013, 5039, 5091, 5130, 5179, 5213, 5303, 5373, 5490, 5502, 5518, 5573] 

acurácia: 0.2

0.20588235294117646

### Trailer do Homem de Ferro

In [10]:
roda_experimento_histograma_global('homem_de_ferro_3_trailer.mp4', 24, cortes_reais['homem_de_ferro'], 
                                  soma_das_diferencas, limiar_histograma_global)

0% 1% 2% 2% 3% 4% 5% 6% 7% 7% 8% 9% 10% 11% 12% 12% 13% 14% 15% 16% 16% 17% 18% 19% 20% 21% 21% 22% 23% 24% 25% 26% 26% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 36% 36% 37% 38% 39% 40% 41% 41% 42% 43% 44% 45% 46% 46% 47% 48% 49% 50% 50% 51% 52% 53% 54% 55% 55% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 65% 65% 66% 67% 68% 69% 70% 70% 71% 72% 73% 74% 75% 75% 76% 77% 78% 79% 80% 80% 81% 82% 83% 84% 84% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 99% 99% 100%

 quadros reais: [33, 98, 136, 231, 242, 300, 327, 350, 356, 381, 404, 433, 449, 483, 503, 582, 601, 647, 659, 683, 699, 714, 738, 756, 786, 812, 837, 856, 871, 904, 918, 953, 984, 1043, 1070, 1114, 1131, 1147, 1158, 1185, 1192, 1214, 1229, 1245, 1259, 1268, 1279, 1288, 1296, 1312, 1337, 1360, 1385, 1401, 1419, 1437, 1449, 1458, 1470, 1510, 1523, 1595, 1639, 1666, 1679, 1701, 1724, 1775, 1800, 1833, 1941, 2023, 2117, 2212, 2276, 2315, 2520, 2558, 2686, 2708] 

acurácia: 0.72 

quadros preditos corretamente: [12, 84, 2

0.725

### Mario Kart

In [11]:
roda_experimento_histograma_global('marioKartTour360p.mp4', 30, cortes_reais['mario_kart_tour'], 
                                  soma_das_diferencas, limiar_histograma_global)

0% 1% 2% 3% 4% 5% 6% 7% 9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 99% 100%
nenhum quadro de corte foi predito


### Ponto de Federer

In [12]:
roda_experimento_histograma_global('pontoFederer_360p.mp4', 25, cortes_reais['pontos_federer'], 
                                  soma_das_diferencas, limiar_histograma_global)

0% 1% 2% 2% 3% 4% 5% 6% 6% 7% 8% 9% 10% 10% 11% 12% 13% 14% 14% 15% 16% 17% 18% 18% 19% 20% 21% 22% 23% 23% 24% 25% 26% 27% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 35% 36% 37% 38% 39% 39% 40% 41% 42% 43% 43% 44% 45% 46% 47% 48% 48% 49% 50% 51% 52% 52% 53% 54% 55% 56% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 64% 65% 66% 67% 68% 69% 69% 70% 71% 72% 73% 73% 74% 75% 76% 77% 77% 78% 79% 80% 81% 81% 82% 83% 84% 85% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [20, 154, 178, 436, 668, 821, 927, 936, 947, 1039, 1486, 1553, 1765, 1887, 2949] 

acurácia: 0.80 

quadros preditos corretamente: [13, 138, 163, 413, 663, 813, 913, 1038, 1463, 1538, 1763, 1863] 

quadros preditos incorretamente: []


0.8

### Coração Valente

In [13]:
roda_experimento_histograma_global('coracaoValenteDiscurso_360p.mp4', 25, cortes_reais['coracao_valente'], 
                                  soma_das_diferencas, limiar_histograma_global)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 9% 10% 10% 11% 11% 12% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 85% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [23, 37, 108, 197, 338, 459, 528, 562, 587, 643, 708, 742, 787, 820, 982, 1100, 1125, 1237, 1315, 1378, 1436, 1470, 1512, 1570, 1625, 1671, 1733, 1877, 1942, 2073, 2120, 2400, 2428, 

0.47761194029850745

### Trailer de Warcraft

In [14]:
roda_experimento_histograma_global('warcraftTrailler360p.mp4', 24, cortes_reais['warcraft_trailler'], 
                                  soma_das_diferencas, limiar_histograma_global)

0% 1% 1% 2% 3% 4% 4% 5% 6% 7% 7% 8% 9% 10% 10% 11% 12% 13% 13% 14% 15% 16% 16% 17% 18% 19% 19% 20% 21% 22% 22% 23% 24% 25% 25% 26% 27% 28% 28% 29% 30% 31% 31% 32% 33% 34% 34% 35% 36% 37% 37% 38% 39% 40% 40% 41% 42% 43% 43% 44% 45% 46% 47% 47% 48% 49% 50% 50% 51% 52% 53% 53% 54% 55% 56% 56% 57% 58% 59% 59% 60% 61% 62% 62% 63% 64% 65% 65% 66% 67% 68% 68% 69% 70% 71% 71% 72% 73% 74% 74% 75% 76% 77% 77% 78% 79% 80% 80% 81% 82% 83% 83% 84% 85% 86% 86% 87% 88% 89% 89% 90% 91% 92% 92% 93% 94% 95% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [120, 153, 178, 196, 236, 252, 285, 300, 362, 400, 430, 470, 504, 547, 579, 621, 628, 672, 702, 717, 727, 740, 760, 768, 782, 802, 874, 899, 925, 952, 1003, 1039, 1080, 1115, 1147, 1198, 1222, 1262, 1304, 1340, 1353, 1375, 1393, 1408, 1439, 1470, 1494, 1521, 1562, 1580, 1606, 1624, 1636, 1661, 1715, 1758, 1775, 1800, 1829, 1867, 1894, 1959, 1967, 1987, 2004, 2024, 2040, 2084, 2112, 2119, 2136, 2160, 2188, 2219, 2239, 2265, 2273, 2291, 2304, 2326, 2358, 23

0.6285714285714286

### Rei Leão

In [15]:
roda_experimento_histograma_global('reiLeao_360p.mp4', 30, cortes_reais['rei_leao'], 
                                  soma_das_diferencas, limiar_histograma_global)

0% 0% 1% 2% 2% 3% 4% 4% 5% 6% 6% 7% 7% 8% 9% 9% 10% 11% 11% 12% 13% 13% 14% 15% 15% 16% 16% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 34% 34% 35% 36% 36% 37% 37% 38% 39% 39% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 52% 52% 53% 53% 54% 55% 55% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 75% 75% 76% 76% 77% 78% 78% 79% 80% 80% 81% 82% 82% 83% 83% 84% 85% 85% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [30, 679, 801, 976, 1027, 1087, 1251, 1301, 1351, 1392, 1563, 1669, 1779, 1815, 1897, 2088, 2279, 2371, 2508, 2535, 2565, 2632, 2728, 2847, 3122, 3229, 3286, 3413, 3649, 3711, 3997, 4117, 4536, 4689] 

acurácia: 0.85 

quadros preditos corretamente: [15, 675, 795, 975, 1005, 1275, 1335, 1545, 1665, 1755, 1785, 1875, 2085, 2265, 2355, 2505, 2535, 2625, 2715, 2835, 

0.8529411764705882

## 2. Histograma Local - Particionamento grid 5x5

In [7]:
def aplicar_histograma_local_com_grid_em_video(video, fps, quadro_limite=None):
    proximo_frame = (fps + 1) // 2
    histogramas = [[[] for i in range(10)] for j in range(10)]
    cap = cv.VideoCapture(video)
    total_de_frames = cap.get(7)
    
    cap.set(1, 1)
    ret, frame = cap.read()
    dimensoes_do_grid = [frame.shape[0] // 5, frame.shape[1] // 5]
    progresso = 0
    while(proximo_frame < total_de_frames):
        cap.set(1, proximo_frame)
        ret, frame = cap.read()
        if(ret == False):
            break
        for i in range(5):
            for j in range(5):
                i_limite_inferior = i*dimensoes_do_grid[0]
                i_limite_superior = i_limite_inferior + dimensoes_do_grid[0] + 1
                j_limite_inferior = j*dimensoes_do_grid[1]
                j_limite_superior = j_limite_inferior + dimensoes_do_grid[1] + 1
                grid_atual = frame[i_limite_inferior:i_limite_superior, j_limite_inferior:j_limite_superior]
                histogramas[i][j].append((proximo_frame, histograma_global(grid_atual)))
        progresso = int(proximo_frame / total_de_frames * 100)
        print(f'{progresso}%', end=' ')
        proximo_frame += fps
        if(quadro_limite != None):
            if(proximo_frame >= quadro_limite):
                cap.release()
                cv.destroyAllWindows()
                print("parou no quadro limite")
                return histogramas
    cap.release()
    cv.destroyAllWindows()
    print('100%')
    return histogramas

def get_quadros_de_corte_do_histograma_local_com_grid(histogramas, distancia, limiar):
    quadros_de_corte = []
    for k in range(len(histogramas[0][0]) - 1):
        corte_encontrado = False
        for i in range(5):
            if(corte_encontrado):
                break
            for j in range(5):
                bgr_histogramas1 = histogramas[i][j][k][1]
                bgr_histogramas2 = histogramas[i][j][k + 1][1]
                if(existe_corte_entre_bgr_histogramas(bgr_histogramas1, bgr_histogramas2, distancia, limiar)):
                    quadros_de_corte.append((histogramas[i][j][k][0]))
                    corte_encontrado = True
                    break
    return quadros_de_corte

def roda_experimento_histograma_local_com_grid(video, fps, quadros_de_corte_reais, distancia, limiar, quadro_limite=None):
    histogramas = aplicar_histograma_local_com_grid_em_video(video, fps, quadro_limite=quadro_limite)
    quadros_de_corte_preditos = get_quadros_de_corte_do_histograma_local_com_grid(histogramas, distancia, limiar)
    return mostrar_resultados_da_deteccao_de_cenas(quadros_de_corte_reais, quadros_de_corte_preditos, fps)

## Detecção de cenas

In [17]:
limiar_histograma_local_com_grid = 11000

### Titanic

In [18]:
roda_experimento_histograma_local_com_grid('titanic-640x360.mp4', 24, cortes_reais['titanic'], 
                                           soma_das_diferencas, limiar_histograma_local_com_grid)

0% 0% 1% 2% 2% 3% 3% 4% 5% 5% 6% 6% 7% 8% 8% 9% 9% 10% 11% 11% 12% 12% 13% 14% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 21% 22% 23% 23% 24% 24% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 39% 40% 41% 41% 42% 42% 43% 44% 44% 45% 45% 46% 47% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 54% 55% 56% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 63% 64% 65% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 72% 73% 74% 74% 75% 75% 76% 77% 77% 78% 78% 79% 80% 80% 81% 81% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 93% 94% 95% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [124, 332, 376, 513, 563, 636, 669, 685, 723, 788, 819, 920, 966, 1000, 1123, 1384, 1611, 1667, 1820, 2138, 2223, 2305, 2615, 2889, 2990, 3109, 3659] 

acurácia: 1.00 

quadros preditos corretamente: [108, 324, 372, 492, 540, 612, 660, 684, 708, 780, 804, 900, 948, 996, 1116, 1380, 1596, 1644, 1812, 2124, 2220, 2292, 2604, 28

1.0

### Pica pau

In [19]:
roda_experimento_histograma_local_com_grid('pica-pau.mkv', 30, cortes_reais['pica_pau'], 
                                           soma_das_diferencas, limiar_histograma_local_com_grid)

0% 0% 1% 2% 2% 3% 4% 4% 5% 5% 6% 7% 7% 8% 9% 9% 10% 10% 11% 12% 12% 13% 13% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 28% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 40% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 55% 55% 56% 56% 57% 58% 58% 59% 59% 60% 61% 61% 62% 63% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 74% 75% 76% 76% 77% 78% 78% 79% 79% 80% 81% 81% 82% 82% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 91% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 97% 98% 99% 99% 100%

 quadros reais: [304, 914, 940, 1294, 1361, 1406, 1504, 1594, 1819, 1901, 2216, 2260, 2341, 2429, 2628, 2704, 2804, 2860, 2929, 3004, 3160, 3530, 3604, 3654, 3892, 4010, 4260, 4366, 4541, 4616, 4729, 4778] 

acurácia: 1.00 

quadros preditos corretamente: [285, 885, 915, 1275, 1335, 1395, 1485, 1575, 1815, 1875, 2205, 2235, 2325, 2415, 2625, 2685, 2775, 2835, 2925

1.0

### Jornal G1

In [20]:
roda_experimento_histograma_local_com_grid('jornal.mkv', 30, cortes_reais['jornal'], 
                                           soma_das_diferencas, limiar_histograma_local_com_grid)

0% 2% 4% 5% 7% 9% 10% 12% 13% 15% 17% 18% 20% 22% 23% 25% 27% 28% 30% 31% 33% 35% 36% 38% 40% 41% 43% 45% 46% 48% 50% 51% 53% 54% 56% 58% 59% 61% 63% 64% 66% 68% 69% 71% 72% 74% 76% 77% 79% 81% 82% 84% 86% 87% 89% 90% 92% 94% 95% 97% 99% 100%

 quadros reais: [157, 458, 554, 1423, 1454, 1621, 1701] 

acurácia: 1.00 

quadros preditos corretamente: [135, 435, 525, 1395, 1425, 1605, 1695] 

quadros preditos incorretamente: [15, 45, 75, 105, 255, 285, 735, 1275, 1305, 1485, 1515, 1545, 1575, 1755]


1.0

### Entrevista do Will Smith

In [21]:
roda_experimento_histograma_local_com_grid('will_smith_interview360p.mp4', 30, cortes_reais['will_smith'], 
                                          soma_das_diferencas, limiar_histograma_local_com_grid)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 10% 10% 11% 11% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [259, 512, 728, 1648, 1901, 2172, 2357, 2402, 2530, 3098, 3136, 3411, 3738, 4595, 4602, 4640, 4665, 4693, 4771, 4799, 4877, 4906, 5013, 5039, 5091, 5130, 5179, 5213, 5303, 5373, 5490, 5502, 5518, 5573] 

acurácia: 0.9

0.9411764705882353

### Trailer do Homem de Ferro

In [22]:
roda_experimento_histograma_local_com_grid('homem_de_ferro_3_trailer.mp4', 24, cortes_reais['homem_de_ferro'], 
                                          soma_das_diferencas, limiar_histograma_local_com_grid)

0% 1% 2% 2% 3% 4% 5% 6% 7% 7% 8% 9% 10% 11% 12% 12% 13% 14% 15% 16% 16% 17% 18% 19% 20% 21% 21% 22% 23% 24% 25% 26% 26% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 36% 36% 37% 38% 39% 40% 41% 41% 42% 43% 44% 45% 46% 46% 47% 48% 49% 50% 50% 51% 52% 53% 54% 55% 55% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 65% 65% 66% 67% 68% 69% 70% 70% 71% 72% 73% 74% 75% 75% 76% 77% 78% 79% 80% 80% 81% 82% 83% 84% 84% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 99% 99% 100%

 quadros reais: [33, 98, 136, 231, 242, 300, 327, 350, 356, 381, 404, 433, 449, 483, 503, 582, 601, 647, 659, 683, 699, 714, 738, 756, 786, 812, 837, 856, 871, 904, 918, 953, 984, 1043, 1070, 1114, 1131, 1147, 1158, 1185, 1192, 1214, 1229, 1245, 1259, 1268, 1279, 1288, 1296, 1312, 1337, 1360, 1385, 1401, 1419, 1437, 1449, 1458, 1470, 1510, 1523, 1595, 1639, 1666, 1679, 1701, 1724, 1775, 1800, 1833, 1941, 2023, 2117, 2212, 2276, 2315, 2520, 2558, 2686, 2708] 

acurácia: 0.86 

quadros preditos corretamente: [12, 84, 1

0.8625

### Mario Kart

In [23]:
roda_experimento_histograma_local_com_grid('marioKartTour360p.mp4', 30, cortes_reais['mario_kart_tour'], 
                                          soma_das_diferencas, limiar_histograma_local_com_grid)

0% 1% 2% 3% 4% 5% 6% 7% 9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 99% 100%
nenhum quadro de corte foi predito


### Ponto de Federer

In [24]:
roda_experimento_histograma_local_com_grid('pontoFederer_360p.mp4', 25, cortes_reais['pontos_federer'], 
                                          soma_das_diferencas, limiar_histograma_local_com_grid)

0% 1% 2% 2% 3% 4% 5% 6% 6% 7% 8% 9% 10% 10% 11% 12% 13% 14% 14% 15% 16% 17% 18% 18% 19% 20% 21% 22% 23% 23% 24% 25% 26% 27% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 35% 36% 37% 38% 39% 39% 40% 41% 42% 43% 43% 44% 45% 46% 47% 48% 48% 49% 50% 51% 52% 52% 53% 54% 55% 56% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 64% 65% 66% 67% 68% 69% 69% 70% 71% 72% 73% 73% 74% 75% 76% 77% 77% 78% 79% 80% 81% 81% 82% 83% 84% 85% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [20, 154, 178, 436, 668, 821, 927, 936, 947, 1039, 1486, 1553, 1765, 1887, 2949] 

acurácia: 1.00 

quadros preditos corretamente: [13, 138, 163, 413, 663, 813, 913, 938, 963, 1038, 1463, 1538, 1763, 1863, 2938] 

quadros preditos incorretamente: [63, 88, 113, 238, 263, 288, 313, 338, 363, 388, 488, 513, 538, 563, 588, 613, 638, 863, 988, 1013, 1513, 1913, 1938, 2113, 2163, 2213, 2263, 2338, 2363, 2413, 2438, 2463, 2513, 2538, 2563, 2588, 2613, 2638, 2713, 2738, 2813, 2838, 2863]


1.0

### Coração Valente

In [25]:
roda_experimento_histograma_local_com_grid('coracaoValenteDiscurso_360p.mp4', 25, cortes_reais['coracao_valente'], 
                                          soma_das_diferencas, limiar_histograma_local_com_grid)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 9% 10% 10% 11% 11% 12% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 85% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [23, 37, 108, 197, 338, 459, 528, 562, 587, 643, 708, 742, 787, 820, 982, 1100, 1125, 1237, 1315, 1378, 1436, 1470, 1512, 1570, 1625, 1671, 1733, 1877, 1942, 2073, 2120, 2400, 2428, 

0.8507462686567164

### Trailer de Warcraft

In [26]:
roda_experimento_histograma_local_com_grid('warcraftTrailler360p.mp4', 24, cortes_reais['warcraft_trailler'], 
                                          soma_das_diferencas, limiar_histograma_local_com_grid)

0% 1% 1% 2% 3% 4% 4% 5% 6% 7% 7% 8% 9% 10% 10% 11% 12% 13% 13% 14% 15% 16% 16% 17% 18% 19% 19% 20% 21% 22% 22% 23% 24% 25% 25% 26% 27% 28% 28% 29% 30% 31% 31% 32% 33% 34% 34% 35% 36% 37% 37% 38% 39% 40% 40% 41% 42% 43% 43% 44% 45% 46% 47% 47% 48% 49% 50% 50% 51% 52% 53% 53% 54% 55% 56% 56% 57% 58% 59% 59% 60% 61% 62% 62% 63% 64% 65% 65% 66% 67% 68% 68% 69% 70% 71% 71% 72% 73% 74% 74% 75% 76% 77% 77% 78% 79% 80% 80% 81% 82% 83% 83% 84% 85% 86% 86% 87% 88% 89% 89% 90% 91% 92% 92% 93% 94% 95% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [120, 153, 178, 196, 236, 252, 285, 300, 362, 400, 430, 470, 504, 547, 579, 621, 628, 672, 702, 717, 727, 740, 760, 768, 782, 802, 874, 899, 925, 952, 1003, 1039, 1080, 1115, 1147, 1198, 1222, 1262, 1304, 1340, 1353, 1375, 1393, 1408, 1439, 1470, 1494, 1521, 1562, 1580, 1606, 1624, 1636, 1661, 1715, 1758, 1775, 1800, 1829, 1867, 1894, 1959, 1967, 1987, 2004, 2024, 2040, 2084, 2112, 2119, 2136, 2160, 2188, 2219, 2239, 2265, 2273, 2291, 2304, 2326, 2358, 23

0.8952380952380953

### Rei Leão

In [27]:
roda_experimento_histograma_local_com_grid('reiLeao_360p.mp4', 30, cortes_reais['rei_leao'], 
                                          soma_das_diferencas, limiar_histograma_local_com_grid)

0% 0% 1% 2% 2% 3% 4% 4% 5% 6% 6% 7% 7% 8% 9% 9% 10% 11% 11% 12% 13% 13% 14% 15% 15% 16% 16% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 34% 34% 35% 36% 36% 37% 37% 38% 39% 39% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 52% 52% 53% 53% 54% 55% 55% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 75% 75% 76% 76% 77% 78% 78% 79% 80% 80% 81% 82% 82% 83% 83% 84% 85% 85% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [30, 679, 801, 976, 1027, 1087, 1251, 1301, 1351, 1392, 1563, 1669, 1779, 1815, 1897, 2088, 2279, 2371, 2508, 2535, 2565, 2632, 2728, 2847, 3122, 3229, 3286, 3413, 3649, 3711, 3997, 4117, 4536, 4689] 

acurácia: 1.00 

quadros preditos corretamente: [15, 675, 795, 975, 1005, 1065, 1245, 1275, 1335, 1365, 1545, 1665, 1755, 1785, 1875, 2085, 2265, 2355, 2505, 2535, 

1.0

## 3. Histograma local - Particionamento Foreground Background (FB)

In [41]:
def aplicar_histograma_local_com_particionamento_FB_em_video(video, fps, quadro_limite=None):
    proximo_frame = (fps + 1) // 2
    histogramas = [[],[],[],[]]
    cap = cv.VideoCapture(video)
    total_de_frames = cap.get(7)
    
    cap.set(1, 1)
    ret, frame = cap.read()
    offsets = [int(frame.shape[0] * 0.1), int(frame.shape[1] * 0.1)]
    progresso = 0
    while(proximo_frame < total_de_frames):
        cap.set(1, proximo_frame)
        ret, frame = cap.read()
        if(ret == False):
            break
        foreground = frame[offsets[0]:, offsets[1]:-offsets[1]]
        histogramas[0].append((proximo_frame, histograma_global(foreground)))
        
        topo_do_background = frame[:offsets[0], offsets[1]:-offsets[1]]
        lado_esquerdo_do_background = frame[:, :offsets[1]]
        lado_direito_do_background =  frame[:, -offsets[1]:]
        histogramas[1].append((proximo_frame, histograma_global(topo_do_background)))
        histogramas[2].append((proximo_frame, histograma_global(lado_esquerdo_do_background)))
        histogramas[3].append((proximo_frame, histograma_global(lado_direito_do_background)))
        
        progresso = int(proximo_frame / total_de_frames * 100)
        print(f'{progresso}%', end=' ')
        proximo_frame += fps
        if(quadro_limite != None):
            if(proximo_frame >= quadro_limite):
                cap.release()
                cv.destroyAllWindows()
                print("parou no quadro limite")
                return histogramas
    cap.release()
    cv.destroyAllWindows()
    print('100%')
    return histogramas

def get_quadros_de_corte_do_histograma_local_com_particionamento_FB(histogramas, distancia, 
                                                                    limiar_foreground, limiar_background):
    quadros_de_corte = []
    limiares = [limiar_foreground, limiar_background]
    for k in range(len(histogramas[0]) - 1):
        if(existe_corte_entre_bgr_histogramas(histogramas[0][k][1], histogramas[0][k + 1][1], distancia, limiar_foreground)):
            quadros_de_corte.append((histogramas[0][k][0]))
        else:
            distancia_background = 0.0
            for i in range(1, 4):
                bgr_histograma1 = histogramas[i][k][1]
                bgr_histograma2 = histogramas[i][k+1][1]
                for banda in range(3):
                    distancia_background += distancia(bgr_histograma1[banda], bgr_histograma2[banda])
            if(distancia_background > limiar_background):
                quadros_de_corte.append((histogramas[0][k][0]))
    return quadros_de_corte

def roda_experimento_histograma_local_com_particionamento_FB(video, fps, quadros_de_corte_reais, distancia, 
                                                             limiar_foreground, limiar_background, quadro_limite=None):
    histogramas = aplicar_histograma_local_com_particionamento_FB_em_video(video, fps, quadro_limite=quadro_limite)
    quadros_de_corte_preditos = get_quadros_de_corte_do_histograma_local_com_particionamento_FB(histogramas, distancia, limiar_foreground, limiar_background)
    return mostrar_resultados_da_deteccao_de_cenas(quadros_de_corte_reais, quadros_de_corte_preditos, fps)

## Detecção de Cenas

In [42]:
limiar_foreground = 180000
limiar_background = 100000

### Titanic

In [43]:
roda_experimento_histograma_local_com_particionamento_FB('titanic-640x360.mp4', 24, cortes_reais['titanic'], 
                                           soma_das_diferencas, limiar_foreground, limiar_background)

0% 0% 1% 2% 2% 3% 3% 4% 5% 5% 6% 6% 7% 8% 8% 9% 9% 10% 11% 11% 12% 12% 13% 14% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 21% 22% 23% 23% 24% 24% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 39% 40% 41% 41% 42% 42% 43% 44% 44% 45% 45% 46% 47% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 54% 55% 56% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 63% 64% 65% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 72% 73% 74% 74% 75% 75% 76% 77% 77% 78% 78% 79% 80% 80% 81% 81% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 93% 94% 95% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [124, 332, 376, 513, 563, 636, 669, 685, 723, 788, 819, 920, 966, 1000, 1123, 1384, 1611, 1667, 1820, 2138, 2223, 2305, 2615, 2889, 2990, 3109, 3659] 

acurácia: 1.00 

quadros preditos corretamente: [108, 324, 372, 492, 540, 612, 660, 684, 708, 780, 804, 900, 948, 996, 1116, 1380, 1596, 1644, 1812, 2124, 2220, 2292, 2604, 28

1.0

### Pica pau

In [45]:
roda_experimento_histograma_local_com_particionamento_FB('pica-pau.mkv', 30, cortes_reais['pica_pau'], 
                                           soma_das_diferencas, limiar_foreground, limiar_background)

0% 0% 1% 2% 2% 3% 4% 4% 5% 5% 6% 7% 7% 8% 9% 9% 10% 10% 11% 12% 12% 13% 13% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 28% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 40% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 55% 55% 56% 56% 57% 58% 58% 59% 59% 60% 61% 61% 62% 63% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 74% 75% 76% 76% 77% 78% 78% 79% 79% 80% 81% 81% 82% 82% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 91% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 97% 98% 99% 99% 100%

 quadros reais: [304, 914, 940, 1294, 1361, 1406, 1504, 1594, 1819, 1901, 2216, 2260, 2341, 2429, 2628, 2704, 2804, 2860, 2929, 3004, 3160, 3530, 3604, 3654, 3892, 4010, 4260, 4366, 4541, 4616, 4729, 4778] 

acurácia: 1.00 

quadros preditos corretamente: [285, 885, 915, 1275, 1335, 1395, 1485, 1575, 1815, 1875, 2205, 2235, 2325, 2415, 2625, 2685, 2775, 2835, 2925

1.0

### Jornal G1

In [46]:
roda_experimento_histograma_local_com_particionamento_FB('jornal.mkv', 30, cortes_reais['jornal'], 
                                           soma_das_diferencas, limiar_foreground, limiar_background)

0% 2% 4% 5% 7% 9% 10% 12% 13% 15% 17% 18% 20% 22% 23% 25% 27% 28% 30% 31% 33% 35% 36% 38% 40% 41% 43% 45% 46% 48% 50% 51% 53% 54% 56% 58% 59% 61% 63% 64% 66% 68% 69% 71% 72% 74% 76% 77% 79% 81% 82% 84% 86% 87% 89% 90% 92% 94% 95% 97% 99% 100%

 quadros reais: [157, 458, 554, 1423, 1454, 1621, 1701] 

acurácia: 1.00 

quadros preditos corretamente: [135, 435, 525, 1395, 1425, 1605, 1695] 

quadros preditos incorretamente: [15, 45, 75, 105, 255, 285, 1275, 1305, 1485, 1515, 1545, 1575, 1755]


1.0

### Entrevista do Will Smith

In [44]:
roda_experimento_histograma_local_com_particionamento_FB('will_smith_interview360p.mp4', 30, cortes_reais['will_smith'], 
                                          soma_das_diferencas, limiar_foreground, limiar_background)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 10% 10% 11% 11% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [259, 512, 728, 1648, 1901, 2172, 2357, 2402, 2530, 3098, 3136, 3411, 3738, 4595, 4602, 4640, 4665, 4693, 4771, 4799, 4877, 4906, 5013, 5039, 5091, 5130, 5179, 5213, 5303, 5373, 5490, 5502, 5518, 5573] 

acurácia: 0.7

0.7647058823529411

### Trailer do Homem de Ferro

In [47]:
roda_experimento_histograma_local_com_particionamento_FB('homem_de_ferro_3_trailer.mp4', 24, cortes_reais['homem_de_ferro'], 
                                          soma_das_diferencas, limiar_foreground, limiar_background)

0% 1% 2% 2% 3% 4% 5% 6% 7% 7% 8% 9% 10% 11% 12% 12% 13% 14% 15% 16% 16% 17% 18% 19% 20% 21% 21% 22% 23% 24% 25% 26% 26% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 36% 36% 37% 38% 39% 40% 41% 41% 42% 43% 44% 45% 46% 46% 47% 48% 49% 50% 50% 51% 52% 53% 54% 55% 55% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 65% 65% 66% 67% 68% 69% 70% 70% 71% 72% 73% 74% 75% 75% 76% 77% 78% 79% 80% 80% 81% 82% 83% 84% 84% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 99% 99% 100%

 quadros reais: [33, 98, 136, 231, 242, 300, 327, 350, 356, 381, 404, 433, 449, 483, 503, 582, 601, 647, 659, 683, 699, 714, 738, 756, 786, 812, 837, 856, 871, 904, 918, 953, 984, 1043, 1070, 1114, 1131, 1147, 1158, 1185, 1192, 1214, 1229, 1245, 1259, 1268, 1279, 1288, 1296, 1312, 1337, 1360, 1385, 1401, 1419, 1437, 1449, 1458, 1470, 1510, 1523, 1595, 1639, 1666, 1679, 1701, 1724, 1775, 1800, 1833, 1941, 2023, 2117, 2212, 2276, 2315, 2520, 2558, 2686, 2708] 

acurácia: 0.88 

quadros preditos corretamente: [12, 84, 1

0.875

### Mario Kart

In [48]:
roda_experimento_histograma_local_com_particionamento_FB('marioKartTour360p.mp4', 30, cortes_reais['mario_kart_tour'], 
                                          soma_das_diferencas, limiar_foreground, limiar_background)

0% 1% 2% 3% 4% 5% 6% 7% 9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 99% 100%

 quadros reais: [31, 213, 262, 308, 361, 410, 459, 508, 559, 606, 708, 804, 901, 1001, 1101, 1198, 1289, 1394, 1445, 1496, 1545, 1592, 1693, 1735, 1772, 1827, 1854, 1939, 1988, 2058, 2081, 2113, 2145, 2170, 2199, 2221, 2245, 2276, 2295, 2316, 2333, 2457, 2732, 2741] 

acurácia: 0.18 

quadros preditos corretamente: [15, 1665, 1725, 1815, 2055, 2085, 2445, 2715] 

quadros preditos incorretamente: []


0.18181818181818182

### Ponto de Federer

In [49]:
roda_experimento_histograma_local_com_particionamento_FB('pontoFederer_360p.mp4', 25, cortes_reais['pontos_federer'], 
                                          soma_das_diferencas, limiar_foreground, limiar_background)

0% 1% 2% 2% 3% 4% 5% 6% 6% 7% 8% 9% 10% 10% 11% 12% 13% 14% 14% 15% 16% 17% 18% 18% 19% 20% 21% 22% 23% 23% 24% 25% 26% 27% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 35% 36% 37% 38% 39% 39% 40% 41% 42% 43% 43% 44% 45% 46% 47% 48% 48% 49% 50% 51% 52% 52% 53% 54% 55% 56% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 64% 65% 66% 67% 68% 69% 69% 70% 71% 72% 73% 73% 74% 75% 76% 77% 77% 78% 79% 80% 81% 81% 82% 83% 84% 85% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [20, 154, 178, 436, 668, 821, 927, 936, 947, 1039, 1486, 1553, 1765, 1887, 2949] 

acurácia: 0.93 

quadros preditos corretamente: [13, 138, 163, 413, 663, 813, 913, 963, 1038, 1463, 1538, 1763, 1863, 2938] 

quadros preditos incorretamente: [63, 113, 338, 363, 388, 538, 563, 588, 638, 1013, 1513, 1913, 2163, 2363, 2838]


0.9333333333333333

### Coração Valente

In [50]:
roda_experimento_histograma_local_com_particionamento_FB('coracaoValenteDiscurso_360p.mp4', 25, cortes_reais['coracao_valente'], 
                                          soma_das_diferencas, limiar_foreground, limiar_background)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 9% 10% 10% 11% 11% 12% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 85% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [23, 37, 108, 197, 338, 459, 528, 562, 587, 643, 708, 742, 787, 820, 982, 1100, 1125, 1237, 1315, 1378, 1436, 1470, 1512, 1570, 1625, 1671, 1733, 1877, 1942, 2073, 2120, 2400, 2428, 

0.6119402985074627

### Trailer de Warcraft

In [51]:
roda_experimento_histograma_local_com_particionamento_FB('warcraftTrailler360p.mp4', 24, cortes_reais['warcraft_trailler'], 
                                          soma_das_diferencas, limiar_foreground, limiar_background)

0% 1% 1% 2% 3% 4% 4% 5% 6% 7% 7% 8% 9% 10% 10% 11% 12% 13% 13% 14% 15% 16% 16% 17% 18% 19% 19% 20% 21% 22% 22% 23% 24% 25% 25% 26% 27% 28% 28% 29% 30% 31% 31% 32% 33% 34% 34% 35% 36% 37% 37% 38% 39% 40% 40% 41% 42% 43% 43% 44% 45% 46% 47% 47% 48% 49% 50% 50% 51% 52% 53% 53% 54% 55% 56% 56% 57% 58% 59% 59% 60% 61% 62% 62% 63% 64% 65% 65% 66% 67% 68% 68% 69% 70% 71% 71% 72% 73% 74% 74% 75% 76% 77% 77% 78% 79% 80% 80% 81% 82% 83% 83% 84% 85% 86% 86% 87% 88% 89% 89% 90% 91% 92% 92% 93% 94% 95% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [120, 153, 178, 196, 236, 252, 285, 300, 362, 400, 430, 470, 504, 547, 579, 621, 628, 672, 702, 717, 727, 740, 760, 768, 782, 802, 874, 899, 925, 952, 1003, 1039, 1080, 1115, 1147, 1198, 1222, 1262, 1304, 1340, 1353, 1375, 1393, 1408, 1439, 1470, 1494, 1521, 1562, 1580, 1606, 1624, 1636, 1661, 1715, 1758, 1775, 1800, 1829, 1867, 1894, 1959, 1967, 1987, 2004, 2024, 2040, 2084, 2112, 2119, 2136, 2160, 2188, 2219, 2239, 2265, 2273, 2291, 2304, 2326, 2358, 23

0.7238095238095238

### Rei Leão

In [52]:
roda_experimento_histograma_local_com_particionamento_FB('reiLeao_360p.mp4', 30, cortes_reais['rei_leao'], 
                                          soma_das_diferencas, limiar_foreground, limiar_background)

0% 0% 1% 2% 2% 3% 4% 4% 5% 6% 6% 7% 7% 8% 9% 9% 10% 11% 11% 12% 13% 13% 14% 15% 15% 16% 16% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 34% 34% 35% 36% 36% 37% 37% 38% 39% 39% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 52% 52% 53% 53% 54% 55% 55% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 75% 75% 76% 76% 77% 78% 78% 79% 80% 80% 81% 82% 82% 83% 83% 84% 85% 85% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [30, 679, 801, 976, 1027, 1087, 1251, 1301, 1351, 1392, 1563, 1669, 1779, 1815, 1897, 2088, 2279, 2371, 2508, 2535, 2565, 2632, 2728, 2847, 3122, 3229, 3286, 3413, 3649, 3711, 3997, 4117, 4536, 4689] 

acurácia: 0.94 

quadros preditos corretamente: [15, 675, 795, 975, 1005, 1065, 1245, 1275, 1335, 1365, 1545, 1665, 1755, 1785, 1875, 2085, 2265, 2355, 2505, 2535, 

0.9411764705882353

## 4. BIC

### BIC paralelizado

In [29]:
def detectar_tipo_de_pixel(janela):
    offsets = [[1,0], [-1,0], [0, 1], [0, -1]]
    for offset in offsets:
        if(abs(janela[1][1][0] - janela[1 + offset[0]][1 + offset[1]][0]) > bic_limiar 
           or abs(janela[1][1][1] - janela[1 + offset[0]][1 + offset[1]][1]) > bic_limiar 
           or abs(janela[1][1][2] - janela[1 + offset[0]][1 + offset[1]][2]) > bic_limiar):
            return 'borda'
    return 'interior'

def fastbic(img):
    histogramas_de_interior = [np.zeros(256, dtype=np.int), np.zeros(256, dtype=np.int), np.zeros(256, dtype=np.int)]
    histogramas_de_borda = [np.zeros(256, dtype=np.int), np.zeros(256, dtype=np.int), np.zeros(256, dtype=np.int)]
    pool = Pool(processes=4)
    for i in range(1, img.shape[0] - 1):
        resultados = pool.map(detectar_tipo_de_pixel, [(img[i-1:i+2,j-1:j+2]) for j in range(1, img.shape[1] - 1 )])
        j = 1
        for tipo_de_pixel in resultados:
            if(tipo_de_pixel == 'interior'):
                histogramas_de_interior[0][img[i][j][0]] += 1
                histogramas_de_interior[1][img[i][j][1]] += 1
                histogramas_de_interior[2][img[i][j][2]] += 1
            else:
                histogramas_de_borda[0][img[i][j][0]] += 1
                histogramas_de_borda[1][img[i][j][1]] += 1
                histogramas_de_borda[2][img[i][j][2]] += 1
            j += 1
    pool.close()
    pool.join()
    return histogramas_de_interior, histogramas_de_borda

In [30]:
def aplicar_bic_em_video(video, fps, limiar=80, quadro_limite=None):
    global bic_limiar
    bic_limiar = limiar
    proximo_frame = (fps + 1) // 2
    caracteristicas = []
    cap = cv.VideoCapture(video)
    total_de_frames = cap.get(7)
    
    cap.set(1, 1)
    ret, frame = cap.read()
    progresso = 0
    while(proximo_frame < total_de_frames):
        cap.set(1, proximo_frame)
        ret, frame = cap.read()
        if(ret == False):
            break
        caracteristicas.append((proximo_frame, fastbic(frame)))
        progresso = int(proximo_frame / total_de_frames * 100)
        print(f'{progresso}%', end=' ')
        proximo_frame += fps
        if(quadro_limite != None):
            if(proximo_frame >= quadro_limite):
                cap.release()
                cv.destroyAllWindows()
                print("bic parou no quadro limite")
                return caracteristicas
                
    cap.release()
    cv.destroyAllWindows()
    print('100%')
    return caracteristicas

def get_quadros_de_corte_do_bic(caracteristicas, distancia, limiar):
    quadros_de_corte = []
    for i in range(len(caracteristicas) - 1):
        ## histogramas de interior e de borda do quadro atual e do próximo
        histogramas1 = caracteristicas[i][1]
        histogramas2 = caracteristicas[i + 1][1]
        if(existe_corte_entre_bgr_histogramas(histogramas1[0], histogramas2[0], distancia, limiar)
           or existe_corte_entre_bgr_histogramas(histogramas1[1], histogramas2[1], distancia, limiar)):
            quadros_de_corte.append(caracteristicas[i][0])
            continue
    return quadros_de_corte

def roda_experimento_bic(video, fps, quadros_de_corte_reais, dLog_limiar, bic_limiar=80, quadro_limite=None):
    warnings.filterwarnings('ignore')
    bic_caracteristicas = aplicar_bic_em_video(video, fps, limiar=bic_limiar, quadro_limite=quadro_limite)
    quadros_de_corte_preditos = get_quadros_de_corte_do_bic(bic_caracteristicas, dLog, dLog_limiar)
    return mostrar_resultados_da_deteccao_de_cenas(quadros_de_corte_reais, quadros_de_corte_preditos, fps)

## Detecção de Cenas

In [31]:
limiar_bic = 800

### Titanic

In [32]:
roda_experimento_bic('titanic-640x360.mp4', 24, cortes_reais['titanic'], limiar_bic)

0% 0% 1% 2% 2% 3% 3% 4% 5% 5% 6% 6% 7% 8% 8% 9% 9% 10% 11% 11% 12% 12% 13% 14% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 21% 22% 23% 23% 24% 24% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 39% 40% 41% 41% 42% 42% 43% 44% 44% 45% 45% 46% 47% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 54% 55% 56% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 63% 64% 65% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 72% 73% 74% 74% 75% 75% 76% 77% 77% 78% 78% 79% 80% 80% 81% 81% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 93% 94% 95% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [124, 332, 376, 513, 563, 636, 669, 685, 723, 788, 819, 920, 966, 1000, 1123, 1384, 1611, 1667, 1820, 2138, 2223, 2305, 2615, 2889, 2990, 3109, 3659] 

acurácia: 0.44 

quadros preditos corretamente: [108, 324, 372, 492, 540, 612, 660, 684, 708, 780, 804, 2604] 

quadros preditos incorretamente: []


0.4444444444444444

### Pica pau

In [33]:
roda_experimento_bic('pica-pau.mkv', 30, cortes_reais['pica_pau'], limiar_bic)

0% 0% 1% 2% 2% 3% 4% 4% 5% 5% 6% 7% 7% 8% 9% 9% 10% 10% 11% 12% 12% 13% 13% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 28% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 40% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 55% 55% 56% 56% 57% 58% 58% 59% 59% 60% 61% 61% 62% 63% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 74% 75% 76% 76% 77% 78% 78% 79% 79% 80% 81% 81% 82% 82% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 91% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 97% 98% 99% 99% 100%

 quadros reais: [304, 914, 940, 1294, 1361, 1406, 1504, 1594, 1819, 1901, 2216, 2260, 2341, 2429, 2628, 2704, 2804, 2860, 2929, 3004, 3160, 3530, 3604, 3654, 3892, 4010, 4260, 4366, 4541, 4616, 4729, 4778] 

acurácia: 0.56 

quadros preditos corretamente: [285, 885, 915, 1275, 1395, 1485, 1815, 2205, 2235, 2325, 2415, 2685, 3135, 4005, 4245, 4515, 4605, 4755] 

qu

0.5625

### Jornal G1

In [34]:
roda_experimento_bic('jornal.mkv', 30, cortes_reais['jornal'], limiar_bic)

0% 2% 4% 5% 7% 9% 10% 12% 13% 15% 17% 18% 20% 22% 23% 25% 27% 28% 30% 31% 33% 35% 36% 38% 40% 41% 43% 45% 46% 48% 50% 51% 53% 54% 56% 58% 59% 61% 63% 64% 66% 68% 69% 71% 72% 74% 76% 77% 79% 81% 82% 84% 86% 87% 89% 90% 92% 94% 95% 97% 99% 100%

 quadros reais: [157, 458, 554, 1423, 1454, 1621, 1701] 

acurácia: 0.57 

quadros preditos corretamente: [135, 435, 525, 1695] 

quadros preditos incorretamente: [45, 75, 105]


0.5714285714285714

### Entrevista do Will Smith

In [35]:
roda_experimento_bic('will_smith_interview360p.mp4', 30, cortes_reais['will_smith'], limiar_bic)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 10% 10% 11% 11% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [259, 512, 728, 1648, 1901, 2172, 2357, 2402, 2530, 3098, 3136, 3411, 3738, 4595, 4602, 4640, 4665, 4693, 4771, 4799, 4877, 4906, 5013, 5039, 5091, 5130, 5179, 5213, 5303, 5373, 5490, 5502, 5518, 5573] 

acurácia: 0.1

0.14705882352941177

### Trailer do Homem de Ferro

In [36]:
roda_experimento_bic('homem_de_ferro_3_trailer.mp4', 24, cortes_reais['homem_de_ferro'], limiar_bic)

0% 1% 2% 2% 3% 4% 5% 6% 7% 7% 8% 9% 10% 11% 12% 12% 13% 14% 15% 16% 16% 17% 18% 19% 20% 21% 21% 22% 23% 24% 25% 26% 26% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 36% 36% 37% 38% 39% 40% 41% 41% 42% 43% 44% 45% 46% 46% 47% 48% 49% 50% 50% 51% 52% 53% 54% 55% 55% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 65% 65% 66% 67% 68% 69% 70% 70% 71% 72% 73% 74% 75% 75% 76% 77% 78% 79% 80% 80% 81% 82% 83% 84% 84% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 99% 99% 100%

 quadros reais: [33, 98, 136, 231, 242, 300, 327, 350, 356, 381, 404, 433, 449, 483, 503, 582, 601, 647, 659, 683, 699, 714, 738, 756, 786, 812, 837, 856, 871, 904, 918, 953, 984, 1043, 1070, 1114, 1131, 1147, 1158, 1185, 1192, 1214, 1229, 1245, 1259, 1268, 1279, 1288, 1296, 1312, 1337, 1360, 1385, 1401, 1419, 1437, 1449, 1458, 1470, 1510, 1523, 1595, 1639, 1666, 1679, 1701, 1724, 1775, 1800, 1833, 1941, 2023, 2117, 2212, 2276, 2315, 2520, 2558, 2686, 2708] 

acurácia: 0.65 

quadros preditos corretamente: [12, 108, 

0.65

### Mario Kart

In [37]:
roda_experimento_bic('marioKartTour360p.mp4', 30, cortes_reais['mario_kart_tour'], limiar_bic)

0% 1% 2% 3% 4% 5% 6% 7% 9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 99% 100%

 quadros reais: [31, 213, 262, 308, 361, 410, 459, 508, 559, 606, 708, 804, 901, 1001, 1101, 1198, 1289, 1394, 1445, 1496, 1545, 1592, 1693, 1735, 1772, 1827, 1854, 1939, 1988, 2058, 2081, 2113, 2145, 2170, 2199, 2221, 2245, 2276, 2295, 2316, 2333, 2457, 2732, 2741] 

acurácia: 0.61 

quadros preditos corretamente: [15, 285, 345, 525, 555, 585, 885, 975, 1365, 1485, 1605, 1665, 1725, 1815, 1845, 1935, 2055, 2085, 2145, 2175, 2205, 2265, 2295, 2325, 2355, 2445, 2715] 

quadros preditos incorretamente: [1635]


0.6136363636363636

### Ponto de Federer

In [38]:
roda_experimento_bic('pontoFederer_360p.mp4', 25, cortes_reais['pontos_federer'], limiar_bic)

0% 1% 2% 2% 3% 4% 5% 6% 6% 7% 8% 9% 10% 10% 11% 12% 13% 14% 14% 15% 16% 17% 18% 18% 19% 20% 21% 22% 23% 23% 24% 25% 26% 27% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 35% 36% 37% 38% 39% 39% 40% 41% 42% 43% 43% 44% 45% 46% 47% 48% 48% 49% 50% 51% 52% 52% 53% 54% 55% 56% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 64% 65% 66% 67% 68% 69% 69% 70% 71% 72% 73% 73% 74% 75% 76% 77% 77% 78% 79% 80% 81% 81% 82% 83% 84% 85% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [20, 154, 178, 436, 668, 821, 927, 936, 947, 1039, 1486, 1553, 1765, 1887, 2949] 

acurácia: 0.67 

quadros preditos corretamente: [13, 163, 413, 663, 913, 1038, 1463, 1538, 1763, 1863] 

quadros preditos incorretamente: []


0.6666666666666666

### Coração Valente

In [39]:
roda_experimento_bic('coracaoValenteDiscurso_360p.mp4', 25, cortes_reais['coracao_valente'], limiar_bic)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 9% 10% 10% 11% 11% 12% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 85% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [23, 37, 108, 197, 338, 459, 528, 562, 587, 643, 708, 742, 787, 820, 982, 1100, 1125, 1237, 1315, 1378, 1436, 1470, 1512, 1570, 1625, 1671, 1733, 1877, 1942, 2073, 2120, 2400, 2428, 

0.22388059701492538

### Trailer de Warcraft

In [40]:
roda_experimento_bic('warcraftTrailler360p.mp4', 24, cortes_reais['warcraft_trailler'], limiar_bic)

0% 1% 1% 2% 3% 4% 4% 5% 6% 7% 7% 8% 9% 10% 10% 11% 12% 13% 13% 14% 15% 16% 16% 17% 18% 19% 19% 20% 21% 22% 22% 23% 24% 25% 25% 26% 27% 28% 28% 29% 30% 31% 31% 32% 33% 34% 34% 35% 36% 37% 37% 38% 39% 40% 40% 41% 42% 43% 43% 44% 45% 46% 47% 47% 48% 49% 50% 50% 51% 52% 53% 53% 54% 55% 56% 56% 57% 58% 59% 59% 60% 61% 62% 62% 63% 64% 65% 65% 66% 67% 68% 68% 69% 70% 71% 71% 72% 73% 74% 74% 75% 76% 77% 77% 78% 79% 80% 80% 81% 82% 83% 83% 84% 85% 86% 86% 87% 88% 89% 89% 90% 91% 92% 92% 93% 94% 95% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [120, 153, 178, 196, 236, 252, 285, 300, 362, 400, 430, 470, 504, 547, 579, 621, 628, 672, 702, 717, 727, 740, 760, 768, 782, 802, 874, 899, 925, 952, 1003, 1039, 1080, 1115, 1147, 1198, 1222, 1262, 1304, 1340, 1353, 1375, 1393, 1408, 1439, 1470, 1494, 1521, 1562, 1580, 1606, 1624, 1636, 1661, 1715, 1758, 1775, 1800, 1829, 1867, 1894, 1959, 1967, 1987, 2004, 2024, 2040, 2084, 2112, 2119, 2136, 2160, 2188, 2219, 2239, 2265, 2273, 2291, 2304, 2326, 2358, 23

0.5523809523809524

### Rei Leão

In [41]:
roda_experimento_bic('reiLeao_360p.mp4', 30, cortes_reais['rei_leao'], limiar_bic)

0% 0% 1% 2% 2% 3% 4% 4% 5% 6% 6% 7% 7% 8% 9% 9% 10% 11% 11% 12% 13% 13% 14% 15% 15% 16% 16% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 34% 34% 35% 36% 36% 37% 37% 38% 39% 39% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 52% 52% 53% 53% 54% 55% 55% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 75% 75% 76% 76% 77% 78% 78% 79% 80% 80% 81% 82% 82% 83% 83% 84% 85% 85% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [30, 679, 801, 976, 1027, 1087, 1251, 1301, 1351, 1392, 1563, 1669, 1779, 1815, 1897, 2088, 2279, 2371, 2508, 2535, 2565, 2632, 2728, 2847, 3122, 3229, 3286, 3413, 3649, 3711, 3997, 4117, 4536, 4689] 

acurácia: 0.53 

quadros preditos corretamente: [15, 975, 1275, 1335, 1755, 1785, 1875, 2085, 2355, 3105, 3225, 3405, 3645, 3705, 3975, 4095, 4515, 4665] 

quadros 

0.5294117647058824

## 5. BIC Local - Particionamento grid 5x5

In [42]:
def get_quadros_de_corte_do_bic_local(caracteristicas, distancia, limiar):
    quadros_de_corte = []
    for k in range(len(caracteristicas[0][0]) - 1):
        corte_encontrado = False
        for i in range(5):
            if(corte_encontrado):
                break
            for j in range(5):
                histogramas1 = caracteristicas[i][j][k][1]
                histogramas2 = caracteristicas[i][j][k + 1][1]
                if(existe_corte_entre_bgr_histogramas(histogramas1[0], histogramas2[0], distancia, limiar)
                   or existe_corte_entre_bgr_histogramas(histogramas1[1], histogramas2[1], distancia, limiar)):
                    quadros_de_corte.append((caracteristicas[i][j][k][0]))
                    corte_encontrado = True
    return quadros_de_corte

def aplicar_bic_local_em_video(video, fps, limiar=80, quadro_limite=None):
    global bic_limiar
    bic_limiar = limiar
    proximo_frame = (fps + 1) // 2
    caracteristicas = [[[] for i in range(10)] for j in range(10)]
    cap = cv.VideoCapture(video)
    total_de_frames = cap.get(7)
    
    cap.set(1, 1)
    ret, frame = cap.read()
    tamanho_do_grid = [frame.shape[0] // 5, frame.shape[1] // 5] 
    progresso = 0
    while(proximo_frame < total_de_frames):
        cap.set(1, proximo_frame)
        ret, frame = cap.read()
        if(ret == False):
            break
        for i in range(5):
            for j in range(5):
                grid_atual = frame[i*tamanho_do_grid[0]:tamanho_do_grid[0]+1, j*tamanho_do_grid[1]:tamanho_do_grid[1]+1]
                caracteristicas[i][j].append((proximo_frame, fastbic(grid_atual)))
        progresso = int(proximo_frame / total_de_frames * 100)
        print(f'{progresso}%', end=' ')
        proximo_frame += fps
        if(quadro_limite != None):
            if(proximo_frame >= quadro_limite):
                cap.release()
                cv.destroyAllWindows()
                print("bic parou no quadro limite")
                return caracteristicas
    cap.release()
    cv.destroyAllWindows()
    print('100%')
    return caracteristicas

def roda_experimento_bic_local(video, fps, quadros_de_corte_reais, dLog_limiar, bic_limiar=80, quadro_limite=None):
    warnings.filterwarnings('ignore')
    bic_caracteristicas = aplicar_bic_local_em_video(video, fps, limiar=bic_limiar, quadro_limite=quadro_limite)
    quadros_de_corte_preditos = get_quadros_de_corte_do_bic_local(bic_caracteristicas, dLog, dLog_limiar)
    return mostrar_resultados_da_deteccao_de_cenas(quadros_de_corte_reais, quadros_de_corte_preditos, fps)

In [43]:
def bic(img):
    histogramas_de_interior = [np.zeros(256, dtype=np.int), np.zeros(256, dtype=np.int), np.zeros(256, dtype=np.int)]
    histogramas_de_borda = [np.zeros(256, dtype=np.int), np.zeros(256, dtype=np.int), np.zeros(256, dtype=np.int)]
    for i in range(1, img.shape[0] - 1):
        for j in range(1, img.shape[1] - 1):
            tipo_de_pixel = detectar_tipo_de_pixel(img[i-1:i+2,j-1:j+2])
            if(tipo_de_pixel == 'interior'):
                histogramas_de_interior[0][img[i][j][0]] += 1
                histogramas_de_interior[1][img[i][j][1]] += 1
                histogramas_de_interior[2][img[i][j][2]] += 1
            else:
                histogramas_de_borda[0][img[i][j][0]] += 1
                histogramas_de_borda[1][img[i][j][1]] += 1
                histogramas_de_borda[2][img[i][j][2]] += 1
    return histogramas_de_interior, histogramas_de_borda

def aplicar_bic_local_paralelo_em_video(video, fps, limiar=80, quadro_limite=None):
    global bic_limiar
    bic_limiar = limiar
    proximo_frame = (fps + 1) // 2
    caracteristicas = [[[] for i in range(10)] for j in range(10)]
    cap = cv.VideoCapture(video)
    total_de_frames = cap.get(7)
    
    cap.set(1, 1)
    ret, frame = cap.read()
    tamanho_do_grid = [frame.shape[0] // 5, frame.shape[1] // 5] 
    progresso = 0
    while(proximo_frame < total_de_frames):
        cap.set(1, proximo_frame)
        ret, frame = cap.read()
        if(ret == False):
            break
        pool = Pool(processes=4)
        for i in range(5):
            resultados = pool.map(bic, [(frame[i*tamanho_do_grid[0]:tamanho_do_grid[0]+1, j*tamanho_do_grid[1]:tamanho_do_grid[1]+1]) for j in range(5)])
            for j in range(5):
                caracteristicas[i][j].append((proximo_frame, resultados[j]))
        pool.close()
        pool.join()
        progresso = int(proximo_frame / total_de_frames * 100)
        print(f'{progresso}%', end=' ')
        proximo_frame += fps
        if(quadro_limite != None):
            if(proximo_frame >= quadro_limite):
                cap.release()
                cv.destroyAllWindows()
                print("bic parou no quadro limite")
                return caracteristicas
    cap.release()
    cv.destroyAllWindows()
    print('100%')
    return caracteristicas

def roda_experimento_bic_local_paralelo(video, fps, quadros_de_corte_reais, dLog_limiar, bic_limiar=80, quadro_limite=None):
    warnings.filterwarnings('ignore')
    bic_caracteristicas = aplicar_bic_local_paralelo_em_video(video, fps, limiar=bic_limiar, quadro_limite=quadro_limite)
    quadros_de_corte_preditos = get_quadros_de_corte_do_bic_local(bic_caracteristicas, dLog, dLog_limiar)
    return mostrar_resultados_da_deteccao_de_cenas(quadros_de_corte_reais, quadros_de_corte_preditos, fps)

## Detecção de cenas

In [44]:
limiar_bic_local = 500

### Titanic

### BÔNUS: Comparando duas formas de paralelizar o BIC local
### versão 1. Grids rodam em paralelo: cada processo tem sua grid

In [45]:
start_time = time.time()
roda_experimento_bic_local_paralelo('titanic-640x360.mp4', 24, cortes_reais['titanic'], limiar_bic_local)
print("--- %s seconds ---" % (time.time() - start_time))

0% 0% 1% 2% 2% 3% 3% 4% 5% 5% 6% 6% 7% 8% 8% 9% 9% 10% 11% 11% 12% 12% 13% 14% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 21% 22% 23% 23% 24% 24% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 39% 40% 41% 41% 42% 42% 43% 44% 44% 45% 45% 46% 47% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 54% 55% 56% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 63% 64% 65% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 72% 73% 74% 74% 75% 75% 76% 77% 77% 78% 78% 79% 80% 80% 81% 81% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 93% 94% 95% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [124, 332, 376, 513, 563, 636, 669, 685, 723, 788, 819, 920, 966, 1000, 1123, 1384, 1611, 1667, 1820, 2138, 2223, 2305, 2615, 2889, 2990, 3109, 3659] 

acurácia: 0.56 

quadros preditos corretamente: [108, 324, 372, 492, 660, 780, 804, 900, 996, 1596, 2124, 2220, 2604, 2868, 2988] 

quadros preditos incorretamente: [36, 84, 2

### versão 2. Operações paralelas dentro de cada grid: cada grid tem vários processos

In [46]:
start_time = time.time()
roda_experimento_bic_local('titanic-640x360.mp4', 24, cortes_reais['titanic'], 
                           limiar_bic_local, quadro_limite=380)
print("--- %s seconds ---" % (time.time() - start_time))

0% 0% 1% 2% 2% 3% 3% 4% 5% 5% 6% 6% 7% 8% 8% 9% bic parou no quadro limite

 quadros reais: [124, 332, 376, 513, 563, 636, 669, 685, 723, 788, 819, 920, 966, 1000, 1123, 1384, 1611, 1667, 1820, 2138, 2223, 2305, 2615, 2889, 2990, 3109, 3659] 

acurácia: 0.07 

quadros preditos corretamente: [108, 324] 

quadros preditos incorretamente: [36, 84]
--- 59.73082661628723 seconds ---


### Pica pau

In [47]:
roda_experimento_bic_local_paralelo('pica-pau.mkv', 30, cortes_reais['pica_pau'], limiar_bic_local)

0% 0% 1% 2% 2% 3% 4% 4% 5% 5% 6% 7% 7% 8% 9% 9% 10% 10% 11% 12% 12% 13% 13% 14% 15% 15% 16% 17% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 28% 29% 30% 30% 31% 32% 32% 33% 33% 34% 35% 35% 36% 36% 37% 38% 38% 39% 40% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 51% 52% 53% 53% 54% 55% 55% 56% 56% 57% 58% 58% 59% 59% 60% 61% 61% 62% 63% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 74% 75% 76% 76% 77% 78% 78% 79% 79% 80% 81% 81% 82% 82% 83% 84% 84% 85% 86% 86% 87% 87% 88% 89% 89% 90% 91% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 97% 98% 99% 99% 100%

 quadros reais: [304, 914, 940, 1294, 1361, 1406, 1504, 1594, 1819, 1901, 2216, 2260, 2341, 2429, 2628, 2704, 2804, 2860, 2929, 3004, 3160, 3530, 3604, 3654, 3892, 4010, 4260, 4366, 4541, 4616, 4729, 4778] 

acurácia: 0.66 

quadros preditos corretamente: [285, 915, 1275, 1335, 1395, 1485, 1575, 1815, 1875, 2205, 2235, 2325, 2415, 2625, 2685, 2925, 3135, 3585, 400

0.65625

### Jornal G1

In [48]:
roda_experimento_bic_local_paralelo('jornal.mkv', 30, cortes_reais['jornal'], limiar_bic_local)

0% 2% 4% 5% 7% 9% 10% 12% 13% 15% 17% 18% 20% 22% 23% 25% 27% 28% 30% 31% 33% 35% 36% 38% 40% 41% 43% 45% 46% 48% 50% 51% 53% 54% 56% 58% 59% 61% 63% 64% 66% 68% 69% 71% 72% 74% 76% 77% 79% 81% 82% 84% 86% 87% 89% 90% 92% 94% 95% 97% 99% 100%

 quadros reais: [157, 458, 554, 1423, 1454, 1621, 1701] 

acurácia: 1.00 

quadros preditos corretamente: [135, 435, 525, 1395, 1425, 1605, 1695] 

quadros preditos incorretamente: [15, 45, 75, 105, 255, 285, 1485, 1545, 1755]


1.0

### Entrevista do Will Smith

In [49]:
roda_experimento_bic_local_paralelo('will_smith_interview360p.mp4', 30, 
                                    cortes_reais['will_smith'], limiar_bic_local)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 10% 10% 11% 11% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [259, 512, 728, 1648, 1901, 2172, 2357, 2402, 2530, 3098, 3136, 3411, 3738, 4595, 4602, 4640, 4665, 4693, 4771, 4799, 4877, 4906, 5013, 5039, 5091, 5130, 5179, 5213, 5303, 5373, 5490, 5502, 5518, 5573] 

acurácia: 0.5

0.5294117647058824

### Trailer do Homem de Ferro

In [50]:
roda_experimento_bic_local_paralelo('homem_de_ferro_3_trailer.mp4', 24, 
                                    cortes_reais['homem_de_ferro'], limiar_bic_local)

0% 1% 2% 2% 3% 4% 5% 6% 7% 7% 8% 9% 10% 11% 12% 12% 13% 14% 15% 16% 16% 17% 18% 19% 20% 21% 21% 22% 23% 24% 25% 26% 26% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 36% 36% 37% 38% 39% 40% 41% 41% 42% 43% 44% 45% 46% 46% 47% 48% 49% 50% 50% 51% 52% 53% 54% 55% 55% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 65% 65% 66% 67% 68% 69% 70% 70% 71% 72% 73% 74% 75% 75% 76% 77% 78% 79% 80% 80% 81% 82% 83% 84% 84% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 99% 99% 100%

 quadros reais: [33, 98, 136, 231, 242, 300, 327, 350, 356, 381, 404, 433, 449, 483, 503, 582, 601, 647, 659, 683, 699, 714, 738, 756, 786, 812, 837, 856, 871, 904, 918, 953, 984, 1043, 1070, 1114, 1131, 1147, 1158, 1185, 1192, 1214, 1229, 1245, 1259, 1268, 1279, 1288, 1296, 1312, 1337, 1360, 1385, 1401, 1419, 1437, 1449, 1458, 1470, 1510, 1523, 1595, 1639, 1666, 1679, 1701, 1724, 1775, 1800, 1833, 1941, 2023, 2117, 2212, 2276, 2315, 2520, 2558, 2686, 2708] 

acurácia: 0.54 

quadros preditos corretamente: [228, 276,

0.5375

### Mario Kart

In [51]:
roda_experimento_bic_local_paralelo('marioKartTour360p.mp4', 30, 
                                    cortes_reais['mario_kart_tour'], limiar_bic_local)

0% 1% 2% 3% 4% 5% 6% 7% 9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 99% 100%

 quadros reais: [31, 213, 262, 308, 361, 410, 459, 508, 559, 606, 708, 804, 901, 1001, 1101, 1198, 1289, 1394, 1445, 1496, 1545, 1592, 1693, 1735, 1772, 1827, 1854, 1939, 1988, 2058, 2081, 2113, 2145, 2170, 2199, 2221, 2245, 2276, 2295, 2316, 2333, 2457, 2732, 2741] 

acurácia: 0.82 

quadros preditos corretamente: [15, 195, 255, 315, 345, 405, 435, 495, 555, 585, 705, 795, 885, 975, 1215, 1275, 1365, 1425, 1605, 1665, 1725, 1815, 1845, 1935, 1965, 2055, 2085, 2115, 2145, 2175, 2205, 2235, 2265, 2295, 2325, 2355] 

quadros preditos incorretamente: [135, 165, 855, 1245, 1335, 1635, 1905, 2025, 2385]


0.8181818181818182

### Ponto de Federer

In [52]:
roda_experimento_bic_local_paralelo('pontoFederer_360p.mp4', 25, 
                                    cortes_reais['pontos_federer'], limiar_bic_local)

0% 1% 2% 2% 3% 4% 5% 6% 6% 7% 8% 9% 10% 10% 11% 12% 13% 14% 14% 15% 16% 17% 18% 18% 19% 20% 21% 22% 23% 23% 24% 25% 26% 27% 27% 28% 29% 30% 31% 31% 32% 33% 34% 35% 35% 36% 37% 38% 39% 39% 40% 41% 42% 43% 43% 44% 45% 46% 47% 48% 48% 49% 50% 51% 52% 52% 53% 54% 55% 56% 56% 57% 58% 59% 60% 60% 61% 62% 63% 64% 64% 65% 66% 67% 68% 69% 69% 70% 71% 72% 73% 73% 74% 75% 76% 77% 77% 78% 79% 80% 81% 81% 82% 83% 84% 85% 85% 86% 87% 88% 89% 89% 90% 91% 92% 93% 94% 94% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [20, 154, 178, 436, 668, 821, 927, 936, 947, 1039, 1486, 1553, 1765, 1887, 2949] 

acurácia: 0.87 

quadros preditos corretamente: [13, 138, 163, 413, 663, 813, 913, 1038, 1463, 1538, 1763, 1863, 2938] 

quadros preditos incorretamente: [88, 338, 538, 563, 613, 713, 738, 763, 1013, 1513, 1838, 1938, 1963, 2088, 2113, 2138, 2188, 2213, 2238, 2338, 2363, 2488, 2513, 2713, 2838, 2863]


0.8666666666666667

### Coração Valente

In [53]:
roda_experimento_bic_local_paralelo('coracaoValenteDiscurso_360p.mp4', 25, 
                                    cortes_reais['coracao_valente'], limiar_bic_local)

0% 0% 1% 1% 2% 2% 3% 3% 4% 4% 5% 5% 6% 6% 7% 7% 8% 8% 9% 9% 10% 10% 11% 11% 12% 12% 12% 13% 13% 14% 14% 15% 15% 16% 16% 17% 17% 18% 18% 19% 19% 20% 20% 21% 21% 22% 22% 23% 23% 24% 24% 25% 25% 26% 26% 27% 27% 28% 28% 29% 29% 30% 30% 31% 31% 32% 32% 33% 33% 34% 34% 35% 35% 36% 36% 36% 37% 37% 38% 38% 39% 39% 40% 40% 41% 41% 42% 42% 43% 43% 44% 44% 45% 45% 46% 46% 47% 47% 48% 48% 49% 49% 50% 50% 51% 51% 52% 52% 53% 53% 54% 54% 55% 55% 56% 56% 57% 57% 58% 58% 59% 59% 60% 60% 61% 61% 61% 62% 62% 63% 63% 64% 64% 65% 65% 66% 66% 67% 67% 68% 68% 69% 69% 70% 70% 71% 71% 72% 72% 73% 73% 74% 74% 75% 75% 76% 76% 77% 77% 78% 78% 79% 79% 80% 80% 81% 81% 82% 82% 83% 83% 84% 84% 85% 85% 85% 86% 86% 87% 87% 88% 88% 89% 89% 90% 90% 91% 91% 92% 92% 93% 93% 94% 94% 95% 95% 96% 96% 97% 97% 98% 98% 99% 99% 100%

 quadros reais: [23, 37, 108, 197, 338, 459, 528, 562, 587, 643, 708, 742, 787, 820, 982, 1100, 1125, 1237, 1315, 1378, 1436, 1470, 1512, 1570, 1625, 1671, 1733, 1877, 1942, 2073, 2120, 2400, 2428, 

0.7910447761194029

### Trailer de Warcraft

In [54]:
roda_experimento_bic_local_paralelo('warcraftTrailler360p.mp4', 24, 
                                    cortes_reais['warcraft_trailler'], limiar_bic_local)

0% 1% 1% 2% 3% 4% 4% 5% 6% 7% 7% 8% 9% 10% 10% 11% 12% 13% 13% 14% 15% 16% 16% 17% 18% 19% 19% 20% 21% 22% 22% 23% 24% 25% 25% 26% 27% 28% 28% 29% 30% 31% 31% 32% 33% 34% 34% 35% 36% 37% 37% 38% 39% 40% 40% 41% 42% 43% 43% 44% 45% 46% 47% 47% 48% 49% 50% 50% 51% 52% 53% 53% 54% 55% 56% 56% 57% 58% 59% 59% 60% 61% 62% 62% 63% 64% 65% 65% 66% 67% 68% 68% 69% 70% 71% 71% 72% 73% 74% 74% 75% 76% 77% 77% 78% 79% 80% 80% 81% 82% 83% 83% 84% 85% 86% 86% 87% 88% 89% 89% 90% 91% 92% 92% 93% 94% 95% 95% 96% 97% 98% 98% 99% 100%

 quadros reais: [120, 153, 178, 196, 236, 252, 285, 300, 362, 400, 430, 470, 504, 547, 579, 621, 628, 672, 702, 717, 727, 740, 760, 768, 782, 802, 874, 899, 925, 952, 1003, 1039, 1080, 1115, 1147, 1198, 1222, 1262, 1304, 1340, 1353, 1375, 1393, 1408, 1439, 1470, 1494, 1521, 1562, 1580, 1606, 1624, 1636, 1661, 1715, 1758, 1775, 1800, 1829, 1867, 1894, 1959, 1967, 1987, 2004, 2024, 2040, 2084, 2112, 2119, 2136, 2160, 2188, 2219, 2239, 2265, 2273, 2291, 2304, 2326, 2358, 23

0.44761904761904764

### Rei Leão

In [55]:
roda_experimento_bic_local_paralelo('reiLeao_360p.mp4', 30, cortes_reais['rei_leao'], limiar_bic_local)

0% 0% 1% 2% 2% 3% 4% 4% 5% 6% 6% 7% 7% 8% 9% 9% 10% 11% 11% 12% 13% 13% 14% 15% 15% 16% 16% 17% 18% 18% 19% 20% 20% 21% 22% 22% 23% 23% 24% 25% 25% 26% 27% 27% 28% 29% 29% 30% 30% 31% 32% 32% 33% 34% 34% 35% 36% 36% 37% 37% 38% 39% 39% 40% 41% 41% 42% 43% 43% 44% 45% 45% 46% 46% 47% 48% 48% 49% 50% 50% 51% 52% 52% 53% 53% 54% 55% 55% 56% 57% 57% 58% 59% 59% 60% 60% 61% 62% 62% 63% 64% 64% 65% 66% 66% 67% 68% 68% 69% 69% 70% 71% 71% 72% 73% 73% 74% 75% 75% 76% 76% 77% 78% 78% 79% 80% 80% 81% 82% 82% 83% 83% 84% 85% 85% 86% 87% 87% 88% 89% 89% 90% 90% 91% 92% 92% 93% 94% 94% 95% 96% 96% 97% 98% 98% 99% 99% 100%

 quadros reais: [30, 679, 801, 976, 1027, 1087, 1251, 1301, 1351, 1392, 1563, 1669, 1779, 1815, 1897, 2088, 2279, 2371, 2508, 2535, 2565, 2632, 2728, 2847, 3122, 3229, 3286, 3413, 3649, 3711, 3997, 4117, 4536, 4689] 

acurácia: 0.59 

quadros preditos corretamente: [975, 1005, 1065, 1245, 1365, 1545, 1755, 1785, 1875, 2265, 2355, 2505, 2835, 3105, 3405, 3645, 3705, 3975, 4095, 45

0.5882352941176471